In [1]:
import pandas as pd 
import json 
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf 
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub 
import tensorflow_text as text
from tensorflow.keras import optimizers

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mrulay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_excel(r'C:\Users\Mrulay\OneDrive - University of Windsor\uWindsor\COMP 8700 - Intro to AI\Project\data\emotionDetection\train.xlsx', index_col=False)
df

,Sentences,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [5]:
stop = stopwords.words('english')

def preProcessText(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '') 
    text = text.replace('@', '')
    text = ' '.join(word.lower() for word in text.split() if word not in stop)
    return text

In [6]:
df['Sentences'] = df['Sentences'].apply(preProcessText)

In [7]:
labels = pd.get_dummies(df['Emotion'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['Sentences'],labels)

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", trainable=False)
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

In [14]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(6, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [17]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

learning_rate = 0.0001
optimizer = optimizers.Adam(learning_rate)
    
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=METRICS)

es = EarlyStopping(
    monitor='accuracy',
    patience=2,
    min_delta = 0.1,
    mode='max'
)

In [18]:
model.fit(X_train, y_train, epochs=20, verbose=1, shuffle=True, callbacks=[es])

Epoch 1/20
375/375 [==============================] - 1948s 5s/step - loss: 0.4435 - accuracy: 0.8130 - precision: 0.1693 - recall: 0.0312
Epoch 2/20
375/375 [==============================] - 1997s 5s/step - loss: 0.4069 - accuracy: 0.8330 - precision: 0.4657 - recall: 0.0136
Epoch 3/20
317/375 [========================>.....] - ETA: 5:23 - loss: 0.4038 - accuracy: 0.8332 - precision: 0.4886 - recall: 0.0170

KeyboardInterrupt: 